# Imports

In [ ]:
!pip install torch transformers datasets

In [ ]:
import json
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, load_metric

# Load dataset from JSON

In [ ]:
poems_path = 'data/dlk_poems_texts.json'
dataset = load_dataset('json', data_files=poems_path, field='poems', split='train')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('benjamin/gerpt2')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length=768
def tokenize(data):
    return tokenizer(data['text'], padding='max_length', truncation=True)

In [ ]:
dataset = dataset.map(tokenize, batched=True)
dataset = dataset.add_column('labels', dataset['input_ids'])

In [ ]:
train_test = dataset.train_test_split(test_size=0.1, shuffle=True)
train_data = train_test['train']
test_data = train_test['test']

# Set up pre-trained model

In [ ]:
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
model = AutoModelForCausalLM.from_pretrained('benjamin/gerpt2')

In [ ]:
perplexity = load_metric('perplexity')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Run fine-tuning

In [ ]:
training_args = TrainingArguments(
    output_dir='models/gerpt2-dlk',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train(resume_from_checkpoint=True)

In [ ]:
trainer.save_model()

## Utilities

In [ ]:
dir(trainer)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_reserved()

In [ ]:
!nvidia-smi

In [ ]:
trainer.save_model()